In [1]:
!pip install transformers datasets evaluate peft bitsandbytes -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.4 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 21.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 27.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.7 MB/s eta 0:00:00:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━

In [2]:
import sys

modules_to_remove = [
    'tensorflow',
    'tensorflow.python',
    'tensorflow.python.pywrap_tensorflow',
    'tensorboard',
]

for module in modules_to_remove:
    if module in sys.modules.keys():
        del sys.modules[module]
        print(f"Removed {module}")

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

In [4]:
from datasets import load_dataset

raw_dataset = load_dataset("dany0407/eli5_category", split="train[:5000]")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/98.6M [00:00<?, ?B/s]

data/validation1-00000-of-00001.parquet:   0%|          | 0.00/7.92M [00:00<?, ?B/s]

data/validation2-00000-of-00001.parquet:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/6.09M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/91772 [00:00<?, ? examples/s]

Generating validation1 split:   0%|          | 0/5446 [00:00<?, ? examples/s]

Generating validation2 split:   0%|          | 0/2375 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5411 [00:00<?, ? examples/s]

In [5]:
raw_dataset = raw_dataset.train_test_split(test_size=0.2)

raw_dataset = raw_dataset.flatten()

In [6]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [7]:
def set_seed(seed):
    import random, numpy as np
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [8]:
set_seed(42)

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_checkpoint = "Qwen/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, device_map='auto', 
                             quantization_config=bnb_config,
                             trust_remote_code=True)

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

In [ ]:
print(model.named_modules())

TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj", "qkv_proj", "gate_proj", "down_proj", "up_proj"]

In [ ]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=TARGET_MODULES,
    bias='none',
    task_type=TaskType.CAUSAL_LM
)

In [ ]:
from peft import get_peft_model

model = get_peft_model(model, lora_config)

In [ ]:
def count_trainable(m):
    t = sum(p.numel() for p in m.parameters() if p.requires_grad)
    tot = sum(p.numel() for p in m.parameters())
    return t, tot, t / tot

In [ ]:
trainable, total, frac = count_trainable(model)

print(f"Trainable: {trainable:,} | Total: {total:,} | Fraction: {frac:.4%}")

In [ ]:
def make_text(example):
    text = example['answers.text']
    return {'text': text}

ds = ds.map(make_text, remove_columns=ds.column_names)

In [ ]:
def tokenize_fn(examples):
    return tokenizer(examples['text'], truncation="max_length", max_length=1024, padding=False)

tokenized_dataset = raw_dataset.map(
    tokenize_fn,
    batched=True,
    num_proc=4,
    remove_columns=raw_dataset.column_names['train']
)

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',
    report_to='none',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    eval_strategy='steps',
    save_strategy='steps',
    eval_steps=0.0625,
    seed=42
)

trainer = Trainer(
    args=training_args,
    model=model,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    data_collator=datacollator
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("lora_adapter")

In [ ]:
inference_model = AutoModelForCausalLM.from_pretrained(
    model_checkpoint,
    device_map='auto',
    quantization_config=bnb_config,
    trust_remote_code=True
)

inference_model = get_peft_model(inference_model, lora_config)

inference_model.load_state_dict(
    torch.load("lora_adapter/pytorch_model.bin"),
    strict=False
)

inference_model.eval()

In [ ]:
prompts = [
    'Объясни, как выиграть Всероссийскую Олимпиаду по Искуственному Интеллекту',
    'Назови 3 причины почему гусевой фермер - оптимальное решение для продолжения карьеры'
]
 
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(next(inference_model.parameters()).device)

with torch.no_grad():
    gen = inference_model.generate(
        input_ids,
        max_new_tokens=200,
        do_sample=False,
        temperature=0.0,
        top_p=0.95,
        repetition_penalty=1.05,
        pad_token_id=tokenizer.eos_token_id,
    )

In [ ]:
print(tokenizer.decode(gen[0], skip_special_tokens=True))